In [ ]:
from copy import deepcopy
import logging
import time
import ccxt
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt

from binance import Binance
from huobi   import Huobi
from base    import BaseExchange
from conf    import dev_symbols
from conf    import all_symbols
from conf    import ma1_size, ma2_size, dea_size
from conf    import point_limit
from conf    import cmc_coin_top100
from conf    import huobi_symbols, k15_symbols
from index import calc_sma, calc_ema, calc_macd
from strategy import ThreeFilter

log_format='%(asctime)s -%(module)s-%(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s'
logging.basicConfig(format=log_format, level=logging.WARN)
pd.set_option('precision', 10)

def draw_histogram(s, title, limit=point_limit):
    fig, ax = plt.subplots()
    ax.bar(list(s.keys())[limit:], list(s.values())[limit:], label='MACD')
    
    ax.set_xlabel('Datetime')
    ax.set_ylabel('Value')
    ax.set_title(title)
    ax.legend()
    fig.set_size_inches(18, 10)
    plt.xticks(rotation=270)
    plt.show()

def draw_line(s1, s2, sv, title, limit=point_limit):
    """
    @param s1
    @param s2
    """
    fig, ax = plt.subplots()
    fig.set_size_inches(18, 10)
    ax.plot(list(s1.keys())[limit:], list(s1.values())[limit:], marker='o', label=f'ema {ma1_size}', color='white')
    ax.plot(list(s2.keys())[limit:], list(s2.values())[limit:], marker='o', label=f'ema {ma2_size}', color='yellow')
    ks = [v[0] for v in sv]
    ps = [v[1] for v in sv]
    ax.plot(ks[limit:], ps[limit:], marker='o', label='price', color='green', linewidth=3)
    ax.set_facecolor("black")
    ax.set_title(title)
    ax.legend()
    plt.grid(linestyle='-.')
    plt.xticks(rotation=270)
    plt.show()

def main():
    binance = Binance()
    for symbol in cmc_coin_top100:
        ohlcvs = binance.get_ohlcv(symbol)
        if ohlcvs is None: continue
        closes = [(datetime.fromtimestamp(i[0]/1000).strftime('%m%d %H'), i[4]) for i in ohlcvs]
        
        smas1 = calc_sma(closes, ma1_size)
        smas2 = calc_sma(closes, ma2_size)
        
        emas1 = calc_ema(smas1[ma1_size-1][1], closes, ma1_size)
        emas2 = calc_ema(smas2[ma2_size-1][1], closes, ma2_size)

        dif, dea, macd = calc_macd(emas1, emas2)
        draw_line(emas1, emas2, closes, f'{symbol} EMA')
        draw_histogram(macd, f'{symbol} MACD')
        
def main1():
    huobi = Huobi()
    # for symbol in huobi_symbols:
    for symbol in ['HT/USDT']:
        ohlcvs = huobi.get_ohlcv(symbol, '4h')
        if ohlcvs is None: continue
        closes = [(datetime.fromtimestamp(i[0]/1000).strftime('%m%d %H'), i[4]) for i in ohlcvs]
        
        smas1 = calc_sma(closes, ma1_size)
        smas2 = calc_sma(closes, ma2_size)
        
        
        emas1 = calc_ema(smas1[ma1_size-1][1], closes, ma1_size)
        emas2 = calc_ema(smas2[ma2_size-1][1], closes, ma2_size)

        dif, dea, macd = calc_macd(emas1, emas2)
        draw_line(emas1, emas2, closes, f'{symbol} EMA')
        draw_histogram(macd, f'{symbol} MACD')

def main2():
    binance = Binance()
    filterS = ThreeFilter()
    for symbol in cmc_coin_top100: #k15_symbols:
        if symbol in ('DAI/USDT'):
            continue
        ohlcvs_1d = binance.get_ohlcv(symbol, '1d')
        if ohlcvs_1d is None: continue
        closes_1d = [(datetime.fromtimestamp(i[0]/1000).strftime('%m%d %H'), i[4]) for i in ohlcvs_1d]

        ohlcvs_4h = binance.get_ohlcv(symbol, '4h')
        if ohlcvs_4h is None: continue
        closes_4h = [(datetime.fromtimestamp(i[0]/1000).strftime('%m%d %H'), i[4]) for i in ohlcvs_4h]


        smas1 = calc_sma(closes_1d, ma1_size)
        smas2 = calc_sma(closes_1d, ma2_size)
        if smas1 is None or smas2 is None:
            continue

        emas1 = calc_ema(smas1[ma1_size-1][1], closes_1d, ma1_size)
        emas2 = calc_ema(smas2[ma2_size-1][1], closes_1d, ma2_size)

        dif, dea, macd = calc_macd(emas1, emas2)
#         draw_line(emas1, emas2, closes_1d, f'{symbol} EMA')
#         draw_histogram(macd, f'{symbol} MACD')

        # 计算
        sma1_4h = calc_sma(closes_4h, ma1_size)
        sma2_4h= calc_sma(closes_4h, ma2_size)

        ema1_4h = calc_ema(sma1_4h[ma1_size-1][1], closes_4h, ma1_size)
        ema2_4h = calc_ema(sma2_4h[ma2_size-1][1], closes_4h, ma2_size)
        
        dif_4h, dea_4h, macd_4h = calc_macd(ema1_4h, ema2_4h)
#         draw_line(ema1_4h, ema2_4h, closes_4h, f'{symbol} EMA')
#         draw_histogram(macd_4h, f'{symbol} MACD')


        action, price = filterS.run(macd, closes_4h, ema1_4h, ohlcvs_4h[-1][5])
        if price is None: price=-1
        print(f"{symbol:<12s} 推荐 {price:8.4f} 价格 {action}")


if __name__ == '__main__':
    main2()

In [ ]:
import time

import ccxt

from conf import dev_symbols

api = ccxt.binance()
api.load_markets()
# api.markets['BTC/USDT']


        